<a href="https://colab.research.google.com/github/arampacha/nlp_tools/blob/main/backtranslation_hf_helsinki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating backtranslations

This notebook can be used to generate backtranslations using pretrained models by Helsinki NLP group from huggingface model hub.

Current version is raw and will be improved

TODOs:

- [ ] Choose better `generate` defaults
- [ ] Select strategy for long inputs: split by sentences, split by chunks of sentences upto some number of tokens, more options?
- [ ] Support different input formats: .txt files etc.
- [ ] Option to add noize to backtranslated text
- [ ] Add docstrings

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq transformers sentencepiece fastai

     |████████████████████████████████| 1.8MB 8.1MB/s 
     |████████████████████████████████| 1.2MB 54.7MB/s 
     |████████████████████████████████| 194kB 58.0MB/s 
     |████████████████████████████████| 3.2MB 58.6MB/s 
     |████████████████████████████████| 890kB 41.4MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 


In [1]:
import torch
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from fastcore.all import *
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

### test

In [2]:
tok_enes = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
en2es = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")

In [3]:
input_ids = tok_enes.encode_plus('hello, world', return_tensors='pt').input_ids

In [4]:
out_ids = en2es.generate(input_ids)

In [5]:
out_ids

tensor([[65000,  2119,     2,   372,     3,     0]])

In [6]:
tok_enes.decode(out_ids[0].numpy(), skip_special_tokens=True)

'Hola, mundo.'

In [7]:
tok_esen = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")
es2en = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-en")

In [8]:
out_ids = es2en.generate(out_ids)

In [9]:
tok_esen.decode(out_ids[0].numpy(), skip_special_tokens=True)

'Hello, world.'

## Setup

In [14]:
def get_models(lang1, lang2, device=None):
    device = ifnone(device, torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    tok1 = AutoTokenizer.from_pretrained(f"Helsinki-NLP/opus-mt-{lang1}-{lang2}")
    tok2 = AutoTokenizer.from_pretrained(f"Helsinki-NLP/opus-mt-{lang2}-{lang1}")
    fwd = AutoModelForSeq2SeqLM.from_pretrained(f"Helsinki-NLP/opus-mt-{lang1}-{lang2}").to(device)
    bwd = AutoModelForSeq2SeqLM.from_pretrained(f"Helsinki-NLP/opus-mt-{lang2}-{lang1}").to(device)
    return tok1, tok2, fwd, bwd

In [ ]:
def backtranslate(texts, lang1, lang2, num_beams=1, device=None):
    device = ifnone(device, torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    tok1, tok2, fwd, bwd = get_models(lang1, lang2, device)
    input_ids = tok1.batch_encode_plus(texts, return_tensors='pt', padding=True, max_length=512, truncation=True).input_ids
    output_ids = fwd.generate(input_ids.to(device), num_beams=num_beams)
    res_ids = bwd.generate(output_ids, num_beams=num_beams)
    return [tok2.decode(ids.detach().cpu(), skip_special_tokens=True).strip() for ids in res_ids]

In [ ]:
def generate(df, tok1, tok2, fwd, bwd, bs=16, num_beams=1, device=None):
    res = []
    for idx in tqdm(np.array_split(df.index.to_numpy(), int(np.ceil(len(df)/bs)))):
        texts = df.iloc[idx, 1].to_list()
        res.extend(backtranslate(texts, tok1, tok2, fwd, bwd, num_beams=num_beams, device=device))
    return pd.DataFrame({'text':res})

In [36]:
class Backtranslator:
    """
    Backtranslation generator for text data augmentation
    Uses pretrained models by Helsinki NLP group
    """
    def __init__(self, source_lang, trans_lang, device=None):
        self.device = ifnone(device, torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        self.tok1, self.tok2, self.fwd, self.bwd = get_models(source_lang, trans_lang, self.device)

    def do_one(self, texts, num_beams=1):
        all_gen_kwargs = {
            'max_length': None,
            'min_length': None, 
            'do_sample': None, 
            'early_stopping': None, 
            'num_beams': None, 
            'temperature': None, 
            'top_k': None, 
            'top_p': None, 
            'repetition_penalty': None, 
            'bad_words_ids': None, 
            'bos_token_id': None, 
            'pad_token_id': None, 
            'eos_token_id': None, 
            'length_penalty': None, 
            'no_repeat_ngram_size': None, 
            'encoder_no_repeat_ngram_size': None, 
            'num_return_sequences': None, 
            'decoder_start_token_id': None, 
            'use_cache': None, 
            'num_beam_groups': None, 
            'diversity_penalty': None, 
            'prefix_allowed_tokens_fn': None, 
            'output_attentions': None, 
            'output_hidden_states': None,
            'output_scores': None, 
            'return_dict_in_generate': None,
        }
        gen_kwargs = {
            'do_sample':True,
            'top_p':0.9,
            'repetition_penalty':1.5
        }

        input_ids = self.tok1.batch_encode_plus(texts, return_tensors='pt', padding=True, max_length=512, truncation=True).input_ids
        output_ids = self.fwd.generate(input_ids.to(self.device), num_beams=num_beams, **gen_kwargs)
        res_ids = self.bwd.generate(output_ids, num_beams=num_beams, **gen_kwargs)
        return [self.tok2.decode(ids.detach().cpu(), skip_special_tokens=True).strip() for ids in res_ids]

    def  generate(self, df, text_col=1, bs=32, num_beams=1):
        res = []
        for idx in tqdm(np.array_split(df.index.to_numpy(), int(np.ceil(len(df)/bs)))):
            texts = df.iloc[idx, text_col].to_list()
            res.extend(self.do_one(texts, num_beams=num_beams))
        return pd.DataFrame({'text':res})

In [12]:
text = 'some random text which might go out different'

In [15]:
enes = Backtranslator('en', 'es')

out = enes.do_one([text])
out

['some random text that might come out differently']

In [27]:
def generate_backtranslations(input_fn, lang1:str, lang2:str, output_fn=None, text_col:int=1, num_beams:int=1, bs:int=16, device=None):
    df = pd.read_csv(input_fn)
    btr = Backtranslator(lang1, lang2, device)
    btr_df = btr.generate(df, text_col=text_col, bs=bs, num_beams=num_beams)
    if output_fn: btr_df.to_csv(output_fn)
    return btr_df

## data prep

In [16]:
from fastai.text.all import *

In [17]:
path = untar_data(URLs.IMDB_SAMPLE, dest='./data')
path.ls()

(#1) [Path('data/imdb_sample/texts.csv')]

In [18]:
df = pd.read_csv(path/'texts.csv')
df.head()

df = df[:100]

## es

In [37]:
btr = Backtranslator('en', 'es')
btr_es = btr.generate(df, bs=64)
btr_es.head()

,text
0,"In-beeping-creible! Meg Ryan doesn’t even see her usual adorable pert in this, which usually makes me forgive her superficial schist tickling performance. Hard to believe she was the producer on this dog. Besides Kevin Kline: what kind of suicide trip has he been on? Whoosh... Banzai!!! Finally it was directed by Big Chill boy? It must be a Jonestown - Hollywood style replay. Wooofff!"
1,"In the US film, it is difficult to see that there is no good end to a situation as in the UK case; rather this is something else which has to be treated with each other like they did regarding time of separation between Hindus and Muslims. It seems more likely than not that British people would simply view tensions among religions and were intelligent enough to exploit them until their very ends."
2,"Every once in a while, a movie will come along for such an awful time that I feel compelled to warn people. If I work all my days and can save only one soul from watching this film, how great it is going to be my joy.<br /br/> Where to start my discussion of pain. To begin with there was a musical montage every five minutes. There was no character development. Each persona was a stereotype. We had man curse, fat guy who eats doughnuts, foreign boy fool etc. The script felt like he was written as if being shot.The production value was so incredibly low that I felt like seeing a high school ..."
3,"The name says it all. I saw this movie with my father when he came out and having served in Korea, had a great admiration for the man.The disappointing thing about the film is that only concentrated on a short period of life of men - curiously his whole lifetime would have made such epic bio-pics that it's amazing to imagine costing production.<br/br />Some posters escape from erroneous characteristics about man which are cheap shots.The theme of ""Duty, Honor, Country"" was not just words blastered from the lips of an upper coal officer – it is deep statement of one's total devotion to coun..."
4,"This movie is successful in being one of the most unique movies you have ever seen. However, this comes from the fact that we cannot make heads or tails out of this disaster. It almost seems like a set of challenges established to determine if you are willing to leave the film and give up money just paid for it. If you don’t want to feel despised by this horrible movie and develop an actual sense of pity for actors involved, everyone has seen better days but then realize they really get enough cash to do so and lose compassion for them only as already done for cinema. I can’t go into suffi..."


In [ ]:
# btr_es.to_csv(path/'btr_es.csv')

## de

In [38]:
btr = Backtranslator('en', 'de')
btr_de = btr.generate(df, bs=64)
btr_de.head()

,text
0,"Meg Ryan doesn't even look like her usual pertely lovable self in what usually makes me forgive their flat-bitty acting shit. Hard to believe that she was the producer on this dog. Also Kevin Kline: What kind of suicide journey did his career go through? Whoosh... Banzai!!! Finally, it got done by a guy who staged Big Chill? Must be an repetition of Jonestown - Hollywood style. Wooofff!"
1,"It's a very good thing, but there is no real fault. The act and camera work are all top-notch. Even the music is great although it was mostly early in film when you still feel relatively cheerful. There isn't any true superstar in cast even if several faces will be familiar.The entire actress does an excellent job with this script. But it's hard to watch because there aren't much end of situation like that presented here. Now it's fashionable for British people to blame Hindus and Muslims against each other, then cruelly they split into two countries. There's some merit enough to use them ..."
2,"If I struggle all my days and can save only one soul from the movie, how great will be that joy. <br /><br />Where do you start with a discussion about pain? For beginning there was every five minutes of musical montage. It wasn't character development. Each character is stereotyped. We had an swearer guy eating fat donuts, Goofy foreigners etc. The script felt like it was written because the film was shot. Production value was so incredibly low that it seemed to me as if i were watching a Junior High Video presentation. Have directors, producers etc. seen any movies before? Halestorm alwa..."
3,"I saw this film with my father when he came out and served in Korea. He had great admiration for the man. The disappointing thing about that movie is, it only focuses on a short period of life - interestingly enough, all his lives would have been such an epic biopic to see how much production costs are stingy. <br /> Some posters escape from one's flawed character which were cheap shots.The theme of ""Duty, Honor, Country"" was not just words coming from the lips of some high-quality officer – it is the deep explanation of total devotion by a man towards his country.<BR/>Ironic Peck was the ..."
4,"This film is not just about being in the release of movies. I have a feeling that I've set myself up as some kind of challenge to determine if you are willing to leave this movie and give away what money you were paying for right now. If you don't want to feel relieved, then it will be through these terrible films which develop an authentic sense of compassion on all actors involved; they all saw better days but realize that there was actually quite much cash paid out for them. <br />To lose your sympathy with how you already did so for the film. I can't go enough over those horrible pictu..."


In [ ]:
# btr_de.to_csv(path/'btr_de.csv')

## fr

In [22]:
btr = Backtranslator('en', 'fr')
btr_fr = btr.generate(df, bs=64)
btr_fr.head()

,text
0,"Meg Ryan doesn't even seem like her usual pert even adorable in what, which normally makes me forgive her actor schtick at the tip of the cat. Hard to believe she was the producer on this dog. More Kevin Kline: What kind of suicide trip was her career? Whoosh... Banzai!!! Finally, it was led by the guy who did Big Chill? Must be a Jonestown replay - Hollywood style. Wooofff!"
1,"There are no superstars in the play, but there are no superstars in the case where the English have been against Hindus and Muslims, and then they are cruelly divided into two countries. There is also some merit in this vision, but it is true that no one has forced Hindus and Muslims in the region to be mistaken as they did at the time of the score. It seems more likely that the British have simply seen tensions between religions and have never had the sense of exploiting them for their own purposes."
2,"If I work all my days and I can save one soul from watching this film, how great will be my joy.<br /><br />Where to start my pain discussion. To start, there was a musical montage every five minutes. There was no character development. Each character was a stereotype. We had sworn guys, big guys who eat donuts, goofy stranger, etc. The script felt like it was written as the film was shot. The production value was so low that it seemed like I was watching a high junior video presentation. Have you directors, producers, etc. ever seen a movie before? Halestorm is getting worse and worse wit..."
3,"I watched this film with my father when he came out and served in Korea, he had a great admiration for man. The disappointing thing of this film is that he focuses only on a short period of man's life - interestingly the whole life of man would have made a biopic so epic that it is stunning to imagine the cost of production.<br /><br />Some posters elude the imperfect characteristics of man, which are cheap clichés. The theme of the film ""Duty, Honor, Country"" are not only words blathered from the lips of a high-brown officer - it is the deep statement of a man's total devotion to his coun..."
4,"This film comes to be one of the most unique movies you've seen. However, it comes from the fact that you can't make heads or tails of this mess. It seems almost like a series of challenges put in place to determine whether or not you are willing to get out of the movie and give up the money you just paid. If you don't want to feel slightly you're going to sit through this horrible movie and develop a real feeling of pity for the actors involved, they've all seen better days, but then you realize that they actually paid a little money to do that and you'll lose pity for them as you've alre..."


In [ ]:
# btr_fr.to_csv(path/'btr_fr.csv')

## ru

In [23]:
btr = Backtranslator('en', 'ru')
btr_ru = btr.generate(df, bs=64)
btr_ru.head()

,text
0,"Meg Ryan doesn't even look at himself as usual, loving, which usually makes me forgive her shallow, tickling actor sketch."
1,"There is no real superstar in this film, no face will be familiar. All the acting work with the script. (br.) But it's hard to see (mainly in the film, when all things are relatively fun.) There is no real superstar in the world, although some faces will know. The whole scene works perfectly with the script. (br.) But it's not very good to see because there's no good understanding of the situation, like the one in this story/in this style. At present, you could blame the British who are more clever than their Hindus and Muslims and then brutally divide them into two countries. In this case..."
2,"Every time a movie happens, it's gonna be so terrible that I feel compelled to warn people, if I work all day and I can save one soul from watching this movie, how great my joy will be."
3,"I watched this movie with my father when he came out and served in Korea, he was very impressed by the man, and the disappointing thing about this movie is that he only focuses on a short period of human life -- it's interesting that a person's whole life would make an epic biopic that it's amazing to imagine the cost of production."
4,"This film may be one of the most unique films you've just seen, but it's because you can't make the head or tails of this mess. Almost like a series of tasks that you can decide whether you want to leave the film and give up the money you just paid. If you don't want to feel unattractive, you're gonna sit in this horrible movie and develop a real feeling of pity for the actors, I saw it all when all these games were best, but then you realize that they were actually paid enough money to do it, and you're gonna lose pity for them, like you've already done for the film. I can't go on with th..."


In [24]:
# btr_ru.to_csv(path/'btr_ru.csv')

## nl

In [25]:
btr = Backtranslator('en', 'nl')
btr_nl = btr.generate(df, bs=64)
btr_nl.head()

,text
0,"Meg Ryan doesn't even look her usual pert lovely self in this, which normally forgives me her superficial touch of acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what suicide journey has his career been? Whoosh... Banzai!!! Finally this was directed by the man who did Big Chill? Must be a replay of Jonestown - Hollywood style. Wooofff!"
1,"This is a very well-made film. Acting, scripting and camera work are all first class.The music is also good, although it is usually early in the film, when things are still relatively happy. There are no really superstars in the cast, although different faces will be known. The whole cast does an excellent job with the script.<br /><br />But it is hard to look, because there is no good end to a situation like that one presented. It is now fashionable to accuse the British of placing Hindus and Muslims against each other, and then cruel enough to separate them in two countries. There is als..."
2,"A film that will be so terrible that I feel compelled to warn people. If I work all my days and I can save only one soul from watching this movie, how great will be my joy.<br /><br />Where to start my discussion about pain. To begin, there was a musical editing every five minutes. There was no character development. Each character was a stereotype. We had cursed man, fat guy who eats donuts, goofy foreign man, etc. The script felt like it was written when the movie was recorded. The production value was so incredibly low that it felt like I was watching a junior high video presentation. H..."
3,"The name says it all. I watched this film with my father when it came out and had served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrates on a short period of the man's life - interestingly enough the man would have made his whole life such an epic bio-pic that it is staggering to imagine the cost of production.<br /><br />Some posters avoid the defects of the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" is not only words he has plod out of the lips of a high-scrappy officer - it is the deep expl..."
4,"This film manages to be one of the most unique films you've seen. However, this comes from the fact that you can't make head or tail of this mess. It almost seems like a series of challenges set to determine whether you're willing to walk out of the movie and give the money you just paid for. If you don't want to be tempted, then you'll get a little bit of pity for this horrible movie and then you'll get a real feeling of compassion for the actors involved. They've all seen better days, but then you realize that they've actually got a little bit of money to do this and then it would have b..."


In [26]:
# btr_nl.to_csv(path/'btr_nl.csv')

## generate_backtranslations function

In [28]:
btr_df = generate_backtranslations(path/'texts.csv', 'en', 'es', 'btr_es', bs=64)

In [29]:
btr_df.head()

,text
0,"In-beeping-creible! Meg Ryan doesn’t even see her usual adorable pert in this, which usually makes me forgive her shallow schtick tickling performance. Hard to believe she was the producer in this dog. Besides Kevin Kline: what kind of suicide trip has been in her career? Whoosh... Banzai!!! Finally this was directed by the guy Big Chill did? It must be a Jonestown replay - Hollywood style. Wooofff!"
1,"In the truth movie, in the truth movie, in which people feel very good, in which people feel very good, in which people feel very good, in which people feel great, in which people feel very good, in which people feel great, in which people feel great, in which people feel great, in where people feel great, in where people feel great, in where people feel great, in where people feel very good, in where people feel very good, in where people feel great, in where people feel great, in where people feel great, in where people feel great, in where people feel great, in where people feel great, ..."
2,"Every once in a while a movie will come along for a while that will be so horrible that I feel compelled to warn people. If I work all my days and can save only one soul from watching this movie, the big thing will be my joy.<br /br /> Where to start my discussion of pain. To start with, there was a musical montage every five minutes. There was no character development. Each character was a stereotype. We had a guy curse, fat guy eats doughnuts, weird dumb guy, etc. The script felt like it was written as if the film was being filmed. The value of production was so incredibly low that it fe..."
3,"The name says it all. I saw this film with my father when he came out and having served in Korea, I had a great admiration for the man. The disappointing thing about this film is that it only concentrates on a short period of man's life - which is interesting that all the life of man would have done such an epic biography that it is amazing to imagine the cost of production.<br /br />Some posters elude the wrong characteristics of man, which are cheap shots.The theme of the film ""Duty, Honor, Country"" is not just mere words blastered from the lips of a high-armed officer - it is the profou..."
4,"This movie is successful in being one of the most unique films you’ve ever seen. However, this comes from the fact that you can’t make heads or tails of this disaster. It almost seems like a set of challenges set to determine whether you’re willing or not to leave the movie and give up the money you just paid. If you don’t want to feel disapproved you’ll sit through this horrible movie and develop a real sense of pity for the actors involved, they’ve all seen better days, but then you realize that they were actually paid a little money to do this and you’ll lose pity for them, as you’ve al..."
